In [2]:
from datasets import load_from_disk

/home/suzuki/.conda/envs/bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_from_disk('./wikipedia_ja_20240801')

In [4]:

# dataset のうち，ランダムにな1000件をvalidation，別のランダムな1000件をtest，その他のデータをtrainに分割
dataset_dict = dataset.shuffle(seed=42).train_test_split(test_size=20, shuffle=True)
train_dataset = dataset_dict['train']
valid_and_test_dataset_dict = dataset_dict['test'].train_test_split(test_size=10, shuffle=True)
valid_dataset = valid_and_test_dataset_dict['train']
test_dataset = valid_and_test_dataset_dict['test']

# 改めてDatasetDictを作成
from datasets import DatasetDict
dataset = DatasetDict({'train': train_dataset, 'validation': valid_dataset, 'test': test_dataset})

In [5]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

model_name = "ku-nlp/bart-base-japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [6]:
from transformers import pipeline

generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, max_new_tokens=60)
generated = generator(dataset['train'][0]['text'])
print(generated)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'generated_text': '｜ −  ｜ ｜ − ｜ ｜ 。 ” は 、 日本 の 地名 。 ” この 項目 は 、 の 「 」 を 使用 して い ます 。 ” 。 この 節 の 出典 ： 。 この 項目 で は 、 ２０００ 年 ３ 月 ３１ 日 現在 の データ を 記載 する 。 この'}]


In [7]:
dataset["train"][0]["text"]

'｜１９８９－９０｜｜２１｜｜０｜｜２｜｜０｜｜｜｜｜｜｜｜'

In [8]:
tokenizer.decode(dataset["train"][0]['input_ids'])

'<s> ｜１９８９－９０｜｜２１｜｜０｜｜２｜｜０｜｜｜｜｜｜｜｜</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [9]:
tokenizer.decode(dataset["train"][0]["labels"])

'<s> ＾ ｉ ［ ｃｈ ｉ ｋｙ ｕ ］ ｕ ＃ ｈ ａ ［ ｃｈ ｉ ｋｙ ｕ ］ ｕ ＿ ｋｙ ｕ ［ ｕ ｚ ｅ ］ ｒ ｏ ＿ ｎ ｉ ］ ｊ ｕ ｕ ＃ ｉ ［ ｃｈ ｉ ＿ ｚ ｅ ］ ｒ ｏ ＿ ｎ ｉ ＿ ｚ ｅ ］ ｒ ｏ ＄</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [10]:
exp_name = "mbart_japanese_wikipedia"
output_dir = f"exp/{exp_name}/results"
logging_dir = f"exp/{exp_name}/logs"

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=3,             # 最大3エポックとする
    per_device_train_batch_size=8,  # バッチサイズ
    auto_find_batch_size=True,      # バッチサイズを自動で見つける

    weight_decay=0.01,              # 重み減衰
    learning_rate=2e-5,             # 学習率
    warmup_steps=1000,              # ウォームアップステップ数

    evaluation_strategy="steps",    # 評価はステップごとに行う
    eval_steps=500,                 # 500ステップごとに評価を行う
    # metric_for_best_model="accuracy", # 最良のモデルの評価指標
    # greater_is_better=True,           # 評価指標が大きいほど良い場合はTrue

    output_dir=output_dir,          # モデルの保存先
    save_strategy="steps",          # モデルの保存はエポックごとに行う
    save_total_limit=3,             # 保存するモデルの数)

    logging_dir=logging_dir,        # ログの保存先
    logging_strategy="steps",       # ログの保存はエポックごとに行う
    logging_steps=500,              # 100ステップごとにログを出力する

    load_best_model_at_end=True,    # 最良のモデルを最後にロードする
)

/home/suzuki/.conda/envs/bert/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3, 
    early_stopping_threshold=0.001)

In [13]:
from transformers import Trainer #, DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    # data_collator=data_collator,
    # compute_metrics=compute_metrics,
    # callbacks=[early_stopping],
)

In [15]:
enable_training = True
if enable_training:
    trainer.train(resume_from_checkpoint="exp/mbart_japanese_wikipedia/results/checkpoint-167000")

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
/home/suzuki/.conda/envs/bert/lib/python3.10/site-packages/transformers/trainer.py:3262: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full co

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
enable_model_loading = False
output_dir = f"exp/{exp_name}/results"
# model_path = f"{output_dir}/checkpoint-58940" # 自分の進捗状況
if enable_model_loading:
    model = MBartForConditionalGeneration.from_pretrained(model_path)

In [17]:
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [18]:
split = 'validation'
i = 1
predicted = generator(dataset[split][i]["text"])
print(predicted)
print(f"input: {dataset[split][i]['text']}")
print(f"target: {dataset[split][i]['phones']}")
print(f"predicted: {predicted[0]['generated_text']}")

[{'generated_text': '＾ ｋ ｏ ［ ｋ ｕ ｓ ａ ｉ ｂ ｉ ｒ ｕ ｍ ａ ｎ ｅ ｊ ｉ ｍ ｅ Ｎ ｔ ｏ ｋ ａ ｂ ｕ ｓｈ ｉ ｋ ｉ ｇ ａ ］ ｉ ｓｈ ａ ＄'}]
input: 国際ビルマネジメント株式会社
target: ^ k o [ k u s a i b i r u m a n e j i m e N t o k a b u sh i k i g a ] i sh a $
predicted: ＾ ｋ ｏ ［ ｋ ｕ ｓ ａ ｉ ｂ ｉ ｒ ｕ ｍ ａ ｎ ｅ ｊ ｉ ｍ ｅ Ｎ ｔ ｏ ｋ ａ ｂ ｕ ｓｈ ｉ ｋ ｉ ｇ ａ ］ ｉ ｓｈ ａ ＄


In [20]:
generator("知らない人に道を聞いてみました")

[{'generated_text': '＾ ｓｈ ｉ ［ ｒ ａ ］ ｎ ａ ｉ ＃ ｈ ｉ ［ ｔ ｏ ｎ ｉ ＃ ｍ ｉ ［ ｃｈ ｉ ｏ ＃ ｋ ｉ ［ ｉ ｔ ｅ ＃ ｕ ［ ｍ ａ ］ ｓｈ ｉ ｔ ａ ＄'}]

In [24]:
generator("千二百万七千五百十七円")

[{'generated_text': '＾ ｓ ｅ ］ Ｎ ＃ ｎ ｉ ［ ｈｙ ａ ｋ ｕ ］ ｍ ａ Ｎ ＃ ｓｈ ｉ ［ ｃｌ ｓ ｅ Ｎ ］ ｇ ｏ ＃ ｈ ｉ ［ ｒ ｏ ｍ ａ ］ ｃｌ ｔ ｏ ＄'}]

In [22]:
generator("墾田永年私財法")

[{'generated_text': '＾ ｋ ｏ ［ Ｎ ｄ ｅ Ｎ ｅ ｅ ｎ ｅ Ｎ ｓｈ ｉ ｚ ａ ｉ ］ ｈ ｏ ｏ ＄'}]

In [27]:
generator("橋の端で箸の端を持って走った")

[{'generated_text': '＾ ｈ ａ ［ ｓｈ ｉ ｎ ｏ ］ ｈ ａ ｓｈ ｉ ｄ ｅ ＃ ｈ ａ ］ ｓｈ ｉ ＃ ｎ ｏ ＃ ｔ ａ ［ Ｎ ｏ ＃ ｍ ｏ ］ ｃｌ ｔ ｅ ＄'}]

In [25]:
generator("ポケットモンスター")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'generated_text': '＾ ｐ ｏ ［ ｃｌ ｋ ｅ ｅ ｍ ｏ ］ Ｎ ｓ ｕ ｔ ａ ａ ＄'}]